In [1]:
import os

In [2]:
%pwd

'e:\\Trekathon\\-Early-Fish-Disease-Detection\\notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Trekathon\\-Early-Fish-Disease-Detection'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [9]:
from early_fish_disease_detection.constants import *
from early_fish_disease_detection.utils.common import read_yaml, create_dir, save_json

In [10]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])


    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            model_path='artifacts/training/model.keras',
            training_data='artifacts/data_ingestion/unzipped/Freshwater Fish Disease Aquaculture in south asia',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [13]:
from urllib.parse import urlparse

In [18]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=Path(self.config.training_data) /'test',  # Path to the 'test' folder
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(Path("scores.json"), data=scores)

In [19]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

2025-01-07 19:36:51,955 - Early-Fish-Disease-Detection-logger - INFO - yaml file config\config.yaml read successfully
2025-01-07 19:36:51,961 - Early-Fish-Disease-Detection-logger - INFO - yaml file params.yaml read successfully
2025-01-07 19:36:51,962 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts created successfully
Found 697 images belonging to 7 classes.


c:\Users\ronit\anaconda3\envs\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


44/44 ━━━━━━━━━━━━━━━━━━━━ 34s 774ms/step - accuracy: 0.5466 - loss: 6.3692
2025-01-07 19:37:28,659 - Early-Fish-Disease-Detection-logger - INFO - json file scores.json saved successfully
